<a href="https://colab.research.google.com/github/JAmanOG/AIPrac/blob/main/AIPrac_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

try:
    import apyori
except ImportError:
    !pip install apyori
    import apyori

from apyori import apriori

df = pd.read_csv('Groceries_dataset.csv', parse_dates=['Date'])
df.head()


<ipython-input-3-47c40facb49d>:14: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv('Groceries_dataset.csv', parse_dates=['Date'])


,Member_number,Date,itemDescription
0,1808,2015-07-21,tropical fruit
1,2552,2015-01-05,whole milk
2,2300,2015-09-19,pip fruit
3,1187,2015-12-12,other vegetables
4,3037,2015-02-01,whole milk


In [4]:
df.isnull().any()

,0
Member_number,False
Date,False
itemDescription,False


In [5]:
all_products = df['itemDescription'].unique()
print("Total products: {}".format(len(all_products)))

Total products: 167


In [6]:
def distribution_plot(x, y, name=None, xaxis=None, yaxis=None):
    fig = go.Figure([go.Bar(x=x, y=y)])
    fig.update_layout(
        title_text=name,
        xaxis_title=xaxis,
        yaxis_title=yaxis
    )
    fig.show()

# Example usage
x = df['itemDescription'].value_counts()
x = x.sort_values(ascending=False)
x = x[:10]
distribution_plot(x.index, x.values, name='Top 10 Items', xaxis='Item Description', yaxis='Count')

In [7]:
# Create one-hot encoded columns
one_hot = pd.get_dummies(df['itemDescription'])

# Drop the original 'itemDescription' column
df.drop('itemDescription', inplace=True, axis=1)

# Join the one-hot encoded columns to the original DataFrame
df = df.join(one_hot)

# Display the first few rows of the updated DataFrame
df.head()

,Member_number,Date,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1808,2015-07-21,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,2552,2015-01-05,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
2,2300,2015-09-19,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,1187,2015-12-12,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,3037,2015-02-01,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


In [18]:
records = df.groupby(["Member_number", "Date"])[all_products[:]].sum()

# Define a function to get product names where the value is greater than zero
def get_Pnames(row):
    # Filter columns where values are greater than zero
    products_with_positive_values = [product for product in all_products if row[product] > 0]
    return products_with_positive_values

# Apply the function to each row in the DataFrame
records['Product_names'] = records.apply(get_Pnames, axis=1)

# Display the result
print(records.head())

Member_number  Date      
1000           2014-06-24                    [whole milk, pastry, salty snack]
               2015-03-15    [whole milk, yogurt, sausage, semi-finished br...
               2015-05-27                           [soda, pickled vegetables]
               2015-07-24                       [canned beer, misc. beverages]
               2015-11-25                          [sausage, hygiene articles]
dtype: object


In [19]:
print("Total transactions: {}".format(len(records)))

Total transactions: 14963


In [23]:
x = records.values
x = [[value for value in sub if value != 0] for sub in x]
transactions = x

In [24]:
transactions[0:10]

[['whole milk', 'pastry', 'salty snack'],
 ['whole milk', 'yogurt', 'sausage', 'semi-finished bread'],
 ['soda', 'pickled vegetables'],
 ['canned beer', 'misc. beverages'],
 ['sausage', 'hygiene articles'],
 ['whole milk', 'rolls/buns', 'sausage'],
 ['whole milk', 'soda'],
 ['frankfurter', 'soda', 'whipped/sour cream'],
 ['beef', 'white bread'],
 ['frankfurter', 'curd']]

In [25]:
rules = apriori(transactions, min_support=0.0030, min_confidence=0.05, min_lift=3, min_length=2,target="rules")
association_results = list(rules)

In [30]:
for item in association_results:
    pair = item[0]  # Extract the antecedent and consequent pairs
    items = [x for x in pair]  # Convert the rule items to a list

    # Print the rule
    print("Rule: " + str(items[0]) + " -> " + str(items[1]))

    # Print the support
    print("Support: " + str(item[1]))

    # Print the confidence (assuming it's in the third position of the first element in item[2])
    print("Confidence: " + str(item[2][0][2]))

    # Print the lift (assuming it's in the fourth position of the first element in item[2])
    print("Lift: " + str(item[2][0][3]))

    # Separator for readability
    print("============================================")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.

